In [11]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score, accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from collections import namedtuple
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
import numpy as np
from ipynb.fs.full.evaluation import *
from ipynb.fs.full.Data_Processing import *
from sklearn.model_selection import KFold
import time

### Baseline classification performance on EEG Dataset

Get with 5 fold cross-validation and performance on the test set
- Load the data of each user for all tests 
- Get performance per user - attention, interest, effort
- Get performance per user 

### 1) Load the data

In [12]:
def average(lst): 
    return sum(lst) / len(lst) 

In [13]:
def choose_base_model(model_type, objective, model_name):
    # choose model
    if model_type == 'clf':
        model_dict = {'RF': RandomForestClassifier(random_state=0, n_jobs=-1),
            'NB': GaussianNB(),
            'SVM': LinearSVC(random_state=0, tol=1e-05),
            'LGBM': LGBMClassifier(objective=objective, num_leaves=31, learning_rate=0.1,  random_state=5),
            'LDA': LinearDiscriminantAnalysis()}       
        return model_dict[model_name]
    else: 
        model = LGBMRegressor(num_leaves=31, learning_rate=0.1, random_state=5)
        return model
    

def choose_obj(num_classes):
    if num_classes == 2:
        objective = 'binary'
        return objective
    else: 
        objective = 'multiclass' 
        return objective

In [14]:
def kfold_predict(X,y, model_type, model_name, class_type):
    """
    Method for running 5 fold cross validation based on a given array of tests
    """
    kf= KFold(n_splits = 5, shuffle = True, random_state =  1)
    
    if model_type == 'clf':
        results = {"Accuracy":[], "Precision":[], "Recall":[], "F1 Score Macro":[],
              "F1 Score Micro":[],"Balanced Accuracy":[]}
    else:
        results = {'RMSE':[], 'R2':[]}
        
    total_predictions = []
    total_true = []
    num_classes = 0
    for train_index, test_index in kf.split(X):

        #Train/test split
        X_train, X_test = np.concatenate(X[train_index]), np.concatenate(X[test_index])
        y_train, y_test = np.concatenate(y[train_index]).astype('int'), np.concatenate(y[test_index]).astype('int')
        
        #used for finding the maximum number of classes during the 5 fold cross validation
        if len(set(y_train)) > num_classes:
            num_classes = len(set(y_train))
        
        #Determine whether it will be multi or binary classification
        if class_type == 'binary':
            y_train = convert_to_binary(y_train)
            y_test = convert_to_binary(y_test)
            objective =class_type
        else:
            objective = choose_obj(num_classes) #choose whether it is binary or multi for LGBM's objective
            
        #get size for reporting
        size = len(X_train) + len(X_test)
        
        
            
        model = choose_base_model(model_type, objective, model_name).fit(X_train,y_train)
        y_pred = model.predict(X_test)
        total_predictions.append(y_pred) # keep track of total predictions
        total_true.append(y_test) # keep track of total true for confusion matrix
        r = get_results(y_test, y_pred, model_type) # get dictionary of all results
        
        for key in r: # loop through dictionary to add to all the scores to the results dictionary
            results[key].append(r[key])

    for key in results: # average out the results after 5 fold cross validation
        results[key] = average(results[key])

    return results, np.concatenate(total_predictions), np.concatenate(total_true) , num_classes , size

In [15]:
def save_plots(model_type, y_true, y_pred, user,label, model_name, class_type):
    if model_type == 'clf':
        # plot confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        saved_file = "results/baseline/clf/confusion matrices/k_fold_performance/{2}_User_{0}_{1}_class{3}_.png".format(user,label, model_name, class_type)
        plot_confusion_matrix(cm, set(y_true), normalize=True , file = saved_file)
    if model_type == 'reg':
        saved_file = "results/baseline/reg/y vs y_pred/per user/{2}_User_{0}_{1}_class{3}.png".format(user,label, model_name, class_typ)
        plot_model(y_true, y_pred, user, label,file=saved_file)

In [16]:
def get_results_per_user(all_users, users, labels , bandpass, model_type, multiple, sigma , class_type):
    """
    Method for getting results per user by applying 5 fold cross validation.
    """
    results = []
#     model_names =     ['RF', 'NB', 'LGBM', 'LDA']
    model_names = ['LGBM']
  
    n_epochs = None
    window_size_samples = None
    for model_name in model_names:
        print("Starting with Baseline Model: {0}...".format(model_name))
        for user in users:
            
            for label in labels:
                print("Running - ClassType {3}, Model: {0}, User: {1}, label: {2}".format(model_name, user,label, class_type))
                time_start = time.time()
                test_list = all_users[user] # list of all the tests
                if bandpass == True:
                    X = np.array([filter_sample(np.array(test.iloc[:,:8])) for test in test_list]) # filtered array of all the inputs for each test
                else:
                    X = np.array([np.array(test.iloc[:,:8]) for test in test_list]) # array of all the inputs for each test
                y = np.array([np.array(test[label]) for test in test_list]) # array of given labels for each test           

                # get results
                r, y_pred, y_true, num_classes, size = kfold_predict(X,y, model_type, model_name, class_type)
                duration = time.time() - time_start
                results.append(collate_results(r, user, label, duration, 
                                               num_classes, size, model_type, 
                                               n_epochs, window_size_samples,
                                               model_name, multiple, sigma, bandpass, class_type))

                #save resuls
                save_plots(model_type, y_true, y_pred, user,label, model_name, class_type)
 
    return results 

In [17]:
def get_results_cross_users(labels, all_users_agg, bandpass, model_type, multiple, sigma , class_type):
    """
    Method for getting results cross-user using 5-Fold cross-validation
    """
    results = []
    user = 'cross'
#     model_names =     ['RF', 'NB', 'LGBM', 'LDA']
    model_names = ['LGBM']

    n_epochs = None
    window_size_samples = None
    for model_name in model_names:
        print("Starting with Baseline Model: {0}...".format(model_name))
    
        # get the results cross-user
        for label in labels: #put the inputs and labels in an array for K-fold
            print("Running - Class Type: {3}, Model: {0}, User: {1}, label: {2}".format(model_name, user,label, class_type))
            time_start = time.time()
            if bandpass == True: # filter the channels using bandpass filtering
                X = np.array([np.array(filter_sample(all_users_agg[key].iloc[:, :8] )) for key in all_users_agg]) #array of aggregated inputs per user
            else:
                X = np.array([np.array(all_users_agg[key].iloc[:, :8] ) for key in all_users_agg])
            y = np.array([np.array(all_users_agg[key][label]) for key in all_users_agg]) # array of aggregated labels per user        

            # get results 
            r, y_pred, y_true, num_classes , size = kfold_predict(X,y, model_type, model_name, class_type)
            duration = time.time() - time_start
            results.append(collate_results(r, user, label, duration, 
                                           num_classes, size, model_type, 
                                           n_epochs, window_size_samples, 
                                           model_name, multiple, sigma, bandpass, class_type))

            # save results
            save_plots(model_type, y_true, y_pred, user,label, model_name, class_type)
            
    return results

In [18]:
def get_all_results(model_type, bandpass, eval_type, class_type):
    """
    Method for getting all results per user and across users by applying 5 fold cross validation.
    """
    # Load all the data
    per_user_path = "/cs/home/ybk1/Dissertation/data/saved user and test data/all_tests_EEG_no_agg.pickle"
    all_user_path = "/cs/home/ybk1/Dissertation/data/saved user and test data/all_tests_EEG.pickle"
    all_users_no_agg = load_file(per_user_path) # dictionary of all users , where tests are split out by user without aggregation for per user analysis
    all_users_agg = load_file(all_user_path) # dictionary of all users, where tests are aggregated by user for cross-user analysis
    
    users = all_users_no_agg.keys()
  
    labels = ['attention','interest','effort']
    multiple = None
    sigma = None
    time_start = time.time()
    
    file = "results/baseline/{0}/tabulated/k fold/baselines_5_fold_CV_performance_{1}_bandpass_{2}_class_{3}.csv".format(model_type, eval_type, bandpass, class_type)
    if eval_type == 'per user':
        # get the results per user
        results = get_results_per_user(all_users_no_agg, users, labels, bandpass, model_type, multiple, sigma , class_type)
        results  = pd.DataFrame(results).to_csv(file, index=False)

    elif eval_type == 'cross user':
        #get results cross user
        results = get_results_cross_users(labels, all_users_agg, bandpass,  model_type, multiple, sigma , class_type)
        results  = pd.DataFrame(results).to_csv(file, index=False)

    elif eval_type == 'both':
        results = []
        results.append(pd.DataFrame(get_results_cross_users(labels, all_users_agg, bandpass,  model_type, multiple, sigma , class_type)))
        results.append(pd.DataFrame(get_results_per_user(all_users_no_agg, users, labels, bandpass, model_type, multiple, sigma , class_type)))                     
        results = pd.concat(results).to_csv(file, index=False)
       
  

    print("Time elapsed! {0}".format(time.time() - time_start))
    return results

In [19]:
r = get_all_results('clf', False, 'both', 'multi')
r2 = get_all_results('clf', True, 'both', 'multi')

r3 = pd.concat(r1,r2).to_csv("results/bulk/LGBM_bandpass_test.csv", index=False)

Starting with Baseline Model: LGBM...
Running - Class Type: multi, Model: LGBM, User: cross, label: attention


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/cs/home/ybk1/python/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-261ba826460c>", line 1, in <module>
    r = get_all_results('clf', False, 'both', 'multi')
  File "<ipython-input-18-17d1b0c27ae1>", line 31, in get_all_results
    results.append(pd.DataFrame(get_results_cross_users(labels, all_users_agg, bandpass,  model_type, multiple, sigma , class_type)))
  File "<ipython-input-17-e34259aa2282>", line 26, in get_results_cross_users
    r, y_pred, y_true, num_classes , size = kfold_predict(X,y, model_type, model_name, class_type)
  File "<ipython-input-14-112c1207783f>", line 39, in kfold_predict
    model = choose_base_model(model_type, objective, model_name).fit(X_train,y_train)
  File "/cs/home/ybk1/python/lib/python3.7/site-packages/lightgbm/sklearn.py", line 805, in fit
    callbacks=callbacks)
  File "/cs/home/ybk1/p

KeyboardInterrupt: 